In [60]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick

import seaborn as sns

import numpy as np

import os
sns.set(style="darkgrid")

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [61]:
production_log = pd.read_csv('../input/sensor-data-custom/product_log_data.csv', infer_datetime_format= True)
production_log = production_log.drop("Unnamed: 0",axis=1)
production_log

In [62]:
quality_log = pd.read_csv('../input/sensor-data-custom/quality_log_data.csv')
quality_log = quality_log.drop("Unnamed: 0",axis=1)
quality_log

## Merge Two DataFrames

In [63]:
production_quality_log = pd.merge(production_log, quality_log,  how='left')
production_quality_log

In [64]:
production_quality_log.machine_id.unique()

In [65]:
production_quality_log = production_quality_log.drop("machine_id",axis=1)
production_quality_log

In [66]:
production_quality_log.dtypes

In [67]:
production_quality_log['timestamp'] = production_quality_log['timestamp'].str.replace('_', ' ')
production_quality_log

In [68]:
production_quality_log.dtypes

In [69]:
production_quality_log[production_quality_log['quality']=='nOK']

## Import Sensor Data

In [70]:
from glob import glob
filenames = glob('../input/sensor-data-custom/sensor_data/2021-05-17_*')
list_of_files_with_data = [pd.read_csv(f, delim_whitespace=True, header=None,names=['sensor 1','sensor 2']) for f in filenames] #Read all files
print(len(list_of_files_with_data))

In [71]:
list_of_files_with_data

In [72]:
production_quality_log['sensor data df'] = list_of_files_with_data
production_quality_log

In [73]:
production_quality_log.drop(['product_id'], axis = 1,inplace=True)

In [74]:
prod_quality={'OK': 1,'nOK': 0}
production_quality_log.quality = [prod_quality[item] for item in production_quality_log.quality]

In [75]:
production_quality_log

In [124]:
import seaborn as sns
x=(production_quality_log['sensor data df'][0][:1000])
sns.distplot(production_quality_log['sensor data df'][0])
sns.distplot(x)
plt.title("Distribution of 1000 data readings vs 20480 data readings of sensor  ")
plt.show() #overlapping distribution , so 1000 data points were considered


## Reducing Sensor Datapoints to 1000

In [77]:
production_quality_log['sensor data df'] = production_quality_log['sensor data df'].apply(lambda x:x[:1000])

In [78]:
print(production_quality_log.shape)
print(production_quality_log['sensor data df'].apply(lambda x:x.shape))

So, we see that we have 1656 timestamps and for each timestamp we have 1000 recorded sensor data points

Parsing the Timestamp object to Datetime Type

In [79]:
#import dateutil
#production_quality_log['timestamp'] = production_quality_log['timestamp'].apply(lambda x:dateutil.parser.parse(x))

In [80]:
#check if NA values are present
print(production_quality_log['sensor data df'][::].isna().sum())
print(production_quality_log.isna().sum())

### Variation of Sensor Readings with Time

In [81]:
# For the steps of plotting time series plots we have to calculate the average of the sensor data for each timestamp

# Calculating the average of the sensor readings for each timestamp
avg_sensor_1,avg_sensor_2 = [],[]
for i in range(production_quality_log.shape[0]):
    avg_sensor_1.append(np.average(production_quality_log['sensor data df'][i]['sensor 1']))
    avg_sensor_2.append(np.average(production_quality_log['sensor data df'][i]['sensor 2']))
    

In [82]:
sample_df = pd.DataFrame({'Time':production_quality_log['timestamp'],'Sensor1_Avg':avg_sensor_1,'Sensor2_Avg':avg_sensor_2})

sample_df['sensor1_mov_avg'] = sample_df.Sensor1_Avg.rolling(30).mean() #30 step window moving average for Sensor 1 values
sample_df['sensor2_mov_avg'] = sample_df.Sensor2_Avg.rolling(30).mean() #30 step window moving average for Sensor 2 values
fig_1 = px.line(sample_df, x='Time',y=['Sensor1_Avg','Sensor2_Avg','sensor1_mov_avg','sensor2_mov_avg'],template='plotly_dark')
fig_1.update_layout(
    font_family="Courier New",
    font_color="white",
    title_font_family="Times New Roman",
    title_font_color="White",
    legend_title_font_color="White"
)
fig_1.update_xaxes(title_font_family="Bold",title='TimeStamp')
fig_1.update_yaxes(title_font_family="Bold",title='Sensor Readings')
fig_1.update_layout(title_text='Sensor Readings vs Time', title_x=0.5)
fig_1.show()


In [83]:
sample_df['quality'] = production_quality_log['quality']
fig_2 = px.line(sample_df, x='Time',y=['quality'],template='plotly_dark')
fig_2.update_layout(
    font_family="Courier New",
    font_color="white",
    title_font_family="Times New Roman",
    title_font_color="White",
    legend_title_font_color="White"
)
fig_2.update_xaxes(title_font_family="Bold",title='TimeStamp')
fig_2.update_yaxes(title_font_family="Bold",title='Sensor Readings')
fig_2.update_layout(title_text='Quality of Production Over Time', title_x=0.5)
fig_2.show()


In [84]:
#correlation between quality,sensor avg and moving avg
import seaborn as sns

sns.heatmap(sample_df.corr(), cmap="YlGnBu", annot=True)


As we see from the above correlation plot,sensor 2 has a higher impact on the quality of the product

Let's try to forecast the sensor data for the next 365 days using Time series 

### Using KNN for classification

In [85]:
#reshaping the original dataset for KNN,logistic and Random forest classifier
df_final_time,df_final_quality,sensor1,sensor2 = [],[],[],[]
for x in range(production_quality_log.shape[0]):
    for y in range(production_quality_log['sensor data df'][0].shape[0]):
        df_final_time.append(production_quality_log['timestamp'][x])
        df_final_quality.append(production_quality_log['quality'][x])
        sensor1.append(production_quality_log['sensor data df'][x]['sensor 1'][y])
        sensor2.append(production_quality_log['sensor data df'][x]['sensor 2'][y])

In [86]:
df_final = pd.DataFrame(columns=['timestamp','quality','sensor1','sensor2'])
df_final['timestamp']=df_final_time
df_final['quality']=df_final_quality
df_final['sensor1']=sensor1
df_final['sensor2']=sensor2


In [87]:
print(df_final.shape)


In [88]:
#plotting the co-relation for the actual data
df_final.boxplot(by ='quality', column =['sensor1'], grid = True)
df_final.boxplot(by ='quality', column =['sensor2'], grid = True)

In [89]:
# IQR
Q1_1 = np.percentile(df_final['sensor1'], 25,
                   interpolation = 'midpoint')
 
Q3_1 = np.percentile(df_final['sensor1'], 75,
                   interpolation = 'midpoint')

IQR_1 = Q3_1 - Q1_1

print("Old Shape: ", df_final.shape)


In [90]:
#removing the outliers for sensor 1
# Upper bound
upper_1 = np.where(df_final['sensor1'] >= (Q3_1+1.5*IQR_1))
# Lower bound
lower_1 = np.where(df_final['sensor1'] <= (Q1_1-1.5*IQR_1))



''' Removing the Outliers '''
df_final.drop(upper_1[0], inplace = True)
df_final.drop(lower_1[0], inplace = True)
 
print("New Shape: ", df_final.shape)

In [91]:
#plotting the boxplots for the outlier treated data for sensor1
df_final.boxplot(by ='quality', column =['sensor1'], grid = True)

In [92]:
df_final.head()

In [93]:
from sklearn.model_selection import train_test_split

X = df_final.drop(['quality','timestamp'],axis=1)
y = df_final['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [94]:
X_train.head()

In [95]:
from sklearn.neighbors import KNeighborsClassifier

error_rates = [] #to store the error rate for each iterated value of K

for a in range(1, 5):
    k = a
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    error_rates.append(np.mean(y_test - preds))

plt.figure(figsize=(10, 7))
plt.plot(range(1,5),error_rates,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

As we see the optimal value of the nearest neighbours K = 1, so, KNN is not a suitable model for predicting the qualities 

## Logistic Regression based Model for preicting Quality

In [96]:

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [97]:
model.fit(X_train,np.array(y_train).ravel())
predictions = model.predict(X_test)

In [98]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

Accuracy of Logistic Regression based model :- 94% , F1- score :- 97%

## Random forest based model for predicting Quality

number of estimators(50) i.e , max. number of decision trees = 50

In [99]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=50,oob_score = True)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,np.array(y_train).ravel())

y_pred=clf.predict(X_test)

In [100]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy of the Random forest based Model = 94.4%

In [101]:
df_final.set_index('timestamp',inplace=True)

In [102]:
sample_df.set_index('Time',inplace=True)
#sample_df = sample_df.drop(['sensor1_mov_avg','sensor2_mov_avg'],axis=1)

In [103]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 7
sample_df_1 = sample_df.drop(['sensor2_mov_avg','Sensor1_Avg','Sensor2_Avg','quality'],axis=1)

In [104]:
#sample_df_1 = sample_df_1[:100] # doing time series forecast using last 100 data points 
sample_df_1.plot() 

In [105]:
sample_df_1.dropna(inplace=True)

In [106]:
#checking if the given data is stationary
from statsmodels.tsa.stattools import adfuller
test_result=adfuller(sample_df_1['sensor1_mov_avg'])
def adfuller_test(readings):
    result=adfuller(readings)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary")
    else:
        print("weak evidence against null hypothesis,indicating it is non-stationary ")

adfuller_test(sample_df_1['sensor1_mov_avg'])


In [107]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(sample_df_1['sensor1_mov_avg'])
plt.show()

In [108]:
#plotting partial autocorrelation and correlation
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sample_df_1['sensor1_mov_avg'].dropna(),lags=24,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sample_df_1['sensor1_mov_avg'].dropna(),lags=24,ax=ax2)

In [109]:
#p=2, d=0 , q=1
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(sample_df_1['sensor1_mov_avg'],order=(2,0,8))
model_fit=model.fit()
model_fit.summary()

In [110]:
sample_df_1['forecast']=model_fit.predict(start=8,end=1500,dynamic=True)
sample_df_1[['sensor1_mov_avg','forecast']].plot(figsize=(12,8),title='Sensor 1 Prediction')

In [111]:
sample_df_2 = sample_df.drop(['sensor1_mov_avg','Sensor1_Avg','Sensor2_Avg','quality'],axis=1)

In [112]:
sample_df_2.plot() 

In [113]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sample_df_2['sensor2_mov_avg'].dropna(),lags=24,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sample_df_2['sensor2_mov_avg'].dropna(),lags=24,ax=ax2)

In [114]:
sample_df_2.dropna(inplace=True)

In [115]:
from statsmodels.tsa.arima_model import ARIMA
model=ARIMA(sample_df_2['sensor2_mov_avg'],order=(2,0,8))
model_fit=model.fit()
model_fit.summary()

In [116]:
sample_df_2['forecast']=model_fit.predict(start=8,end=1500,dynamic=True)
sample_df_2[['sensor2_mov_avg','forecast']].plot(figsize=(12,8),title='Sensor 2 Prediction')

As we see that even with the moving average method , auto-correlation for time series analysis for sensors 1 and 2 has been low. Hence, ARIMA based time series is an adequate model here